In [2]:
import zipfile
import os

# Path to your uploaded ZIP file (update if your filename is different)
zip_path = "/content/model_training.zip"

# Destination folder (where files will be extracted)
extract_to = "/content/model_training"

# Create folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files extracted successfully to:", extract_to)

# List extracted contents
for root, dirs, files in os.walk(extract_to):
    level = root.replace(extract_to, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


Files extracted successfully to: /content/model_training
model_training/
  model_training/
    artifacts/
      tfidf_vectorizer.pkl
    data/
      image_embeddings_train.npy
      X_train_tfidf.npz
      y_train_log.npy
      X_test_tfidf.npz
      X_train_combined.npz
      test_numeric_enhanced.csv
      train_numeric_enhanced.csv
      y_train.npy
      X_test_combined.npz
      image_embeddings_test.npy


In [3]:
import numpy as np
from scipy import sparse

# Paths
data_dir = "/content/model_training/model_training/data"

# Load sparse feature matrices
X_train_sparse = sparse.load_npz(f"{data_dir}/X_train_combined.npz")
X_test_sparse  = sparse.load_npz(f"{data_dir}/X_test_combined.npz")

# Load target arrays (log price)
y_train_log = np.load(f"{data_dir}/y_train_log.npy")

print("Data loaded successfully!")
print(f"Train shape: {X_train_sparse.shape}")
print(f"Test  shape: {X_test_sparse.shape}")
print(f"Target shape: {y_train_log.shape}")

# Optional sanity checks
print("\nSample target values:", y_train_log[:5])
print("Mean:", y_train_log.mean(), "Min:", y_train_log.min(), "Max:", y_train_log.max())


Data loaded successfully!
Train shape: (10000, 12057)
Test  shape: (10000, 12057)
Target shape: (10000,)

Sample target values: [1.773256   2.64759223 1.08856195 3.44489524 4.21197944]
Mean: 2.738698719653852 Min: 0.12221763272424921 Max: 6.420939107676499


In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

# --- Train/Validation split ---
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train_sparse, y_train_log, test_size=0.2, random_state=42
)

print("Split completed")
print("Train subset:", X_train_sub.shape, "Validation subset:", X_val.shape)

# --- Helper functions ---
def safe_expm1(x):
    """Inverse log1p safely"""
    x = np.clip(x, -20, 20)  # prevent overflow
    return np.expm1(x)

def smape_from_log(y_true_log, y_pred_log):
    """Compute SMAPE on the price scale"""
    y_true = safe_expm1(y_true_log)
    y_pred = safe_expm1(y_pred_log)
    denominator = np.maximum((np.abs(y_true) + np.abs(y_pred)) / 2, 1e-8)
    smape = np.mean(np.abs(y_true - y_pred) / denominator)
    return smape  # 0–2 range (not %)

print("SMAPE function ready")


Split completed
Train subset: (8000, 12057) Validation subset: (2000, 12057)
SMAPE function ready


In [11]:
import xgboost as xgb
import numpy as np

# Parameters (same tuned values)
params = {
    "objective": "reg:squarederror",
    "eta": 0.025,            # learning_rate
    "max_depth": 9,
    "subsample": 0.85,
    "colsample_bytree": 0.85,
    "alpha": 1.0,            # reg_alpha
    "lambda": 2.5,           # reg_lambda
    "gamma": 0.2,
    "tree_method": "hist",
    "seed": 42,
    "verbosity": 1
}

# Convert to DMatrix (works with CSR sparse)
dtrain = xgb.DMatrix(X_train_sub, label=y_train_sub)
dvalid = xgb.DMatrix(X_val, label=y_val)

# Train with early stopping
num_boost_round = 3000
early_stopping_rounds = 100

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    early_stopping_rounds=early_stopping_rounds,
    verbose_eval=100
)

best_it = bst.best_iteration if hasattr(bst, "best_iteration") else bst.best_ntree_limit
print("Best iteration:", best_it)

[0]	train-rmse:0.93348	valid-rmse:0.93800
[100]	train-rmse:0.46284	valid-rmse:0.78056
[200]	train-rmse:0.34407	valid-rmse:0.76523
[300]	train-rmse:0.26392	valid-rmse:0.76020
[400]	train-rmse:0.19650	valid-rmse:0.75795
[500]	train-rmse:0.15237	valid-rmse:0.75637
[600]	train-rmse:0.14364	valid-rmse:0.75621
[631]	train-rmse:0.14282	valid-rmse:0.75617
Best iteration: 532


In [13]:
# --- Prediction & SMAPE computation for XGBoost >= 1.6 ---
import numpy as np

# Get predictions using best iteration
if 'best_iteration' in locals():
    # modern way: specify iteration range
    val_pred_log = bst.predict(dvalid, iteration_range=(0, best_it))
else:
    # fallback if you don't have best_it variable
    val_pred_log = bst.predict(dvalid)

# Safe inverse transform
def safe_expm1_clip(x, max_log=15.0):
    x = np.clip(x, a_min=-10.0, a_max=max_log)
    return np.expm1(x)

def smape_percent_raw(y_true_raw, y_pred_raw):
    y_true = np.asarray(y_true_raw, dtype=float)
    y_pred = np.asarray(y_pred_raw, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    mask = denom > 0
    frac = np.zeros_like(denom, dtype=float)
    frac[mask] = np.abs(y_true[mask] - y_pred[mask]) / denom[mask]
    return 100.0 * np.mean(frac)

# Convert back to original price scale
y_val_raw = safe_expm1_clip(y_val)
val_pred_raw = safe_expm1_clip(val_pred_log)

# Compute SMAPE (%)
val_smape_pct = smape_percent_raw(y_val_raw, val_pred_raw)
print(f"Validation SMAPE (%): {val_smape_pct:.4f}")


Validation SMAPE (%): 59.2720


In [15]:
# Convert test set to DMatrix
dtest = xgb.DMatrix(X_test_sparse)

In [16]:
import numpy as np

# Predict log(price + 1)
test_pred_log = bst.predict(dtest)

# Convert back to actual price scale safely
test_pred_raw = np.expm1(np.clip(test_pred_log, a_min=-10, a_max=15))

# Ensure all values are positive floats
test_pred_raw = np.maximum(test_pred_raw, 0.0)

print(f"Predictions generated: {len(test_pred_raw)}")
print(f"Prediction range: {test_pred_raw.min():.4f} → {test_pred_raw.max():.4f}")


Predictions generated: 10000
Prediction range: 1.7151 → 139.5893


In [18]:
import pandas as pd
import os

# Path to test file (update if needed)
test_path = "/content/model_training/model_training/data/test_numeric_enhanced.csv"

# Load test IDs or create dummy ones
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
    if "sample_id" in test_df.columns:
        test_ids = test_df["sample_id"]
        print("Loaded sample_id from test_numeric_enhanced.csv")
    else:
        test_ids = np.arange(1, len(test_pred_raw) + 1)
        print("No sample_id column found — using sequential IDs.")
else:
    test_ids = np.arange(1, len(test_pred_raw) + 1)
    print("test_numeric_enhanced.csv not found — using sequential IDs.")

# Build submission DataFrame
submission = pd.DataFrame({
    "sample_id": test_ids,
    "price": test_pred_raw.astype(float)
})

# Save to CSV
output_path = "/content/test_output.csv"
submission.to_csv(output_path, index=False)

print("\nSubmission file saved:", output_path)
print(submission.head())


Loaded sample_id from test_numeric_enhanced.csv

Submission file saved: /content/test_output.csv
   sample_id      price
0     100179  21.914616
1     245611  22.392508
2     146263  18.288519
3      95658   9.255433
4      36806  20.395180
